In [1]:
# Install dependencies if you haven't already
# !pip install openai-whisper torch

import whisper
import torch
import os
from whisper.utils import get_writer

# 1. Check for GPU (CUDA) to enable the "5x-10x speed" mentioned in your README
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"System Check: Using {device.upper()}")

if device == "cuda":
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU not found. Running on CPU (this will be slower).")


System Check: Using CPU
⚠️ GPU not found. Running on CPU (this will be slower).


In [2]:
def generate_srt_advanced(video_path, model_size="large-v3", prompt_context=""):
    """
    Advanced generator using large-v3 and context prompting for better accuracy.
    """
    if not os.path.exists(video_path):
        print("❌ File not found.")
        return

    print(f"1. Loading superior model '{model_size}' (this will take more VRAM)...")
    try:
        # "large-v3" is the best model for Telugu as of 2025/2026
        model = whisper.load_model(model_size, device=device)
    except Exception as e:
        print(f"Error: {e}. Try using 'medium' if your GPU runs out of memory.")
        return

    print("2. Transcribing with enhanced accuracy parameters...")
    print("   (Beam size=5, Temperature=0 for stability)")
    
    # We add an 'initial_prompt' to guide the style.
    # We also use 'beam_size=5' to look for the most probable translation.
    result = model.transcribe(
        video_path, 
        task="translate", 
        language="te",
        initial_prompt=prompt_context, # Crucial for context
        beam_size=5,                   # Checks 5 variations (slower but more accurate)
        best_of=5,
        temperature=0                  # Forces it to be factual, less "creative"
    )
    
    print("3. Saving SRT...")
    output_dir = os.path.dirname(video_path) or "."
    writer = get_writer("srt", output_dir)
    writer(result, video_path)
    
    print(f"✅ Done! Check the folder for your .srt file.")

In [3]:
# --- SETTINGS ---
video_filename = "my_telugu_video.mp4"

# TELL THE AI WHAT THE VIDEO IS ABOUT (Important!)
# Example: "A vlog about cooking Hyderabadi biryani and visiting Charminar."
# Example: "A tech tutorial explaining Python coding in Telugu."
context_hint = "This is a Telugu video about unconditional love, this might have english words as well. Translate sentences to clear English."

# --- RUN ---
# We force 'large-v3' for best results. 
# If your laptop crashes (Out of Memory), change it back to 'medium'.
generate_srt_advanced(video_filename, model_size="large-v3", prompt_context=context_hint)


1. Loading superior model 'large-v3' (this will take more VRAM)...
2. Transcribing with enhanced accuracy parameters...
   (Beam size=5, Temperature=0 for stability)


C:\Users\surya\anaconda3\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


3. Saving SRT...
✅ Done! Check the folder for your .srt file.
